# 📦 Fase 03: Preparación de los Datos (CRISP-DM)


En esta fase preparamos los datos para el modelado. Esto incluye:
- Selección de columnas relevantes.
- Limpieza de datos (valores nulos, outliers).
- Creación de nuevas variables (feature engineering).
- Integración de múltiples fuentes.
- Transformación de datos (escalado, codificación).

Este notebook ejecuta los nodos definidos en `nodes.py` y visualiza los resultados intermedios.


In [1]:

import pandas as pd
from kedro.framework.context import KedroContext
from kedro.framework.session import KedroSession
import matplotlib.pyplot as plt
import seaborn as sns

# Inicializar sesión Kedro
session = KedroSession.create()
context = session.load_context()
catalog = context.catalog


[09/16/25 05:03:15] INFO     Using                                                                  ]8;id=580728;file://C:\Users\amaro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\kedro\framework\project\__init__.py\__init__.py]8;;\:]8;id=109814;file://C:\Users\amaro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\kedro\framework\project\__init__.py#270\270]8;;\
                             'C:\Users\amaro\AppData\Local\Packages\PythonSoftwareFoundation.Python                
                             .3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\                
                             kedro\framework\project\rich_logging.yml' as logging configuration.                   

╭─────────────────────────────── Traceback (most recent call last) ────────────────────────────────╮
│ in <module>:8                                                                                    │
│                                                                                                  │
│    5 import seaborn as sns                                                                       │
│    6                                                                                             │
│    7 # Inicializar sesión Kedro                                                                  │
│ ❱  8 session = KedroSession.create()                                                             │
│    9 context = session.load_context()                                                            │
│   10 catalog = context.catalog                                                                   │
│   11                                                                                             │
│                                                                                                  │
│ C:\Users\amaro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCa │
│ che\local-packages\Python312\site-packages\kedro\framework\session\session.py:156 in create      │
│                                                                                                  │
│   153 │   │   Returns:                                                                           │
│   154 │   │   │   A new ``KedroSession`` instance.                                               │
│   155 │   │   """                                                                                │
│ ❱ 156 │   │   validate_settings()                                                                │
│   157 │   │                                                                                      │
│   158 │   │   session = cls(                                                                     │
│   159 │   │   │   project_path=project_path,                                                     │
│                                                                                                  │
│ C:\Users\amaro\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCa │
│ che\local-packages\Python312\site-packages\kedro\framework\project\__init__.py:332 in            │
│ validate_settings                                                                                │
│                                                                                                  │
│   329 │   More info on the dynaconf issue: https://github.com/dynaconf/dynaconf/issues/460       │
│   330 │   """                                                                                    │
│   331 │   if PACKAGE_NAME is None:                                                               │
│ ❱ 332 │   │   raise ValueError(                                                                  │
│   333 │   │   │   "Package name not found. Make sure you have configured the project using "     │
│   334 │   │   │   "'bootstrap_project'. This should happen automatically if you are using "      │
│   335 │   │   │   "Kedro command line interface."                                                │
╰──────────────────────────────────────────────────────────────────────────────────────────────────╯
ValueError: Package name not found. Make sure you have configured the project using 'bootstrap_project'. This 
should happen automatically if you are using Kedro command line interface.

## 🧹 Limpieza de Datos: `prepare_customers`

In [ ]:

from proyecto_ml.pipelines.data_engineering.nodes import prepare_customers

df_customers = catalog.load("customer_validated")
customers_prepared = prepare_customers(df_customers)
catalog.save("customers_prepared", customers_prepared)
customers_prepared.head()


In [ ]:

# Visualizar distribución de edad y tenure
fig, axes = plt.subplots(1, 2, figsize=(12, 4))
sns.histplot(customers_prepared["age_years"], kde=True, ax=axes[0])
axes[0].set_title("Distribución de Edad")
sns.histplot(customers_prepared["tenure_days"], kde=True, ax=axes[1])
axes[1].set_title("Distribución de Tenure (días)")
plt.tight_layout()
plt.show()


## 📊 Feature Engineering: `compute_rfm`

In [ ]:

from proyecto_ml.pipelines.data_engineering.nodes import compute_rfm

df_purchases = catalog.load("purchase_validated")
rfm_features = compute_rfm(df_purchases)
catalog.save("rfm_features", rfm_features)
rfm_features.head()


## 🛍️ Preferencias de Categoría: `compute_category_preferences`

In [ ]:

from proyecto_ml.pipelines.data_engineering.nodes import compute_category_preferences

df_transactions = catalog.load("transactions_enriched")
category_prefs = compute_category_preferences(df_transactions)
catalog.save("category_prefs", category_prefs)
category_prefs.head()


## 🧩 Integración y Transformación: `assemble_features`

In [ ]:

from proyecto_ml.pipelines.data_engineering.nodes import assemble_features

customer_features = assemble_features(customers_prepared, rfm_features, category_prefs)
catalog.save("customer_features", customer_features)
customer_features.head()


In [ ]:

# Visualizar correlación entre features numéricos
sns.heatmap(customer_features.select_dtypes(include='number').corr(), annot=True, cmap="coolwarm")
plt.title("Correlación entre Features Numéricos")
plt.show()
